## Analýza zamestnancov VŠ

In [2]:
import pandas as pd

from pandasql import sqldf
# from sklearn import datasets

In [3]:
df = pd.read_csv('../data/regzam.csv', index_col=0)
display(df)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_EndDate,Employment_Ratio,Guarantee,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy
0,18799121690,Kočan,Štefan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,NaN,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,NaN,100,False,NaN,NaN,NaN,NaN,NaN
1,14002860472,Korgo,Dušan,DOC,JUDR,PHD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
2,17922733804,Buzalka,Ján,prof.,PhDr.,CSc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
3,16290288493,Stieranka,Jozef,prof.,Ing.,PhD.,Akadémia Policajného zboru,NaN,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,NaN,100,True,"['bezpečnostnoprávna ochrana osôb a majetku', ...","['Akadémia Policajného zboru', 'Akadémia Polic...","[None, None, None, None, None, None, None]","['1', '2', '1', '2', '2', '1', '2']","['1', '1', '2', '2', '2', '3', '3']"
4,21401303068,Wäldl,Ľuboš,doc.,JUDr.,PhD.,Akadémia Policajného zboru,NaN,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,NaN,100,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Spurný,Joža,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,NaN,100,False,NaN,NaN,NaN,NaN,NaN
32726,13500390262,Harčárová,Marcela,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,NaN,50,False,NaN,NaN,NaN,NaN,NaN
32727,17108478442,Juhásová,Jana,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,NaN,50,False,NaN,NaN,NaN,NaN,NaN
32728,15917135755,Kubušová,Miriama,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,NaN,50,False,NaN,NaN,NaN,NaN,NaN


In [4]:
id_set = set(df["Employee_ID"])
print(len(id_set))

32730


-> máme Dataset s 32730 riadkami, pričom žiadne 2 Employee_ID nie sú rovnaké

In [5]:
name_set = set(df["FamilyName"] + " " + df["GivenName"])
len(name_set)

31639

-> 31639 rôznych mien; zostáva zistiť, či pri duplikátnych menách sa jedná o tú istú osobu, alebo nie.
Tiež by nás mohlo zaujímať, ako ukladali osoby, ktorým sa zmenilo meno...

In [6]:
def find_name_duplicates(df):
    name_set = set()
    name_duplicates = set()
    duplicates_ids = set()
    for index, row in df.iterrows():
        name = row["FamilyName"] + " " + row["GivenName"]
        if name in name_set:
            name_duplicates.add(name)
        else:
            name_set.add(name)
    for index, row in df.iterrows():
        name = row["FamilyName"] + " " + row["GivenName"]
        if name in name_duplicates:
            duplicates_ids.add(row["Employee_ID"])
    return tuple(name_duplicates), tuple(duplicates_ids)

In [7]:
name_duplicates, ids = find_name_duplicates(df)

In [8]:
q = f"SELECT * FROM df WHERE Employee_ID IN {ids}"
duplicates_df = sqldf(q, globals()).sort_values(by=['FamilyName', 'GivenName'])
display(duplicates_df)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_EndDate,Employment_Ratio,Guarantee,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy
1327,49910822214,Adamíková,Katarína,None,Mgr.,None,None,None,None,None,None,None,None,0,None,None,None,None,None
1576,18913486096,Adamíková,Katarína,None,Mgr.,None,None,None,None,None,None,None,None,0,None,None,None,None,None
114,39501978150,Adamčík,František,prof.,Ing.,CSc.,Technická univerzita v Košiciach,Letecká fakulta,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2005-02-01,None,100,1,"['prevádzka lietadiel', 'prevádzka lietadiel',...","['Technická univerzita v Košiciach', 'Technick...","['Letecká fakulta', 'Letecká fakulta', 'Leteck...","['1', '1', '2']","['2', '3', '3']"
1325,17240205086,Adamčík,František,None,Ing.,PhD.,None,None,None,None,None,None,None,0,None,None,None,None,None
133,51642863772,Antalová,Jana,None,MUDr.,PhD.,Univerzita Komenského v Bratislave,Lekárska fakulta,Odborný asistent,s vysokoškolským vzdelaním tretieho stupňa,2020-08-15,None,100,0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,15552653387,Žilka,Tomáš,None,MUDr.,None,Slovenská zdravotnícka univerzita v Bratislave,Lekárska fakulta,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,s vysokoškolským vzdelaním druhého stupňa,2022-02-01,None,100,0,None,None,None,None,None
1107,10056354153,Žilková,Viktória,None,Mgr.,None,None,None,None,None,None,None,None,0,None,None,None,None,None
1994,14058627798,Žilková,Viktória,None,None,MSc.,Univerzita Komenského v Bratislave,Lekárska fakulta,Výskumný pracovník - výskumník,s vysokoškolským vzdelaním druhého stupňa,2022-06-15,None,50,0,None,None,None,None,None
530,14639882132,Žuffová,Anna,None,None,None,None,None,None,None,None,None,None,0,None,None,None,None,None


In [9]:
df.query("GivenName == 'Viktória' and FamilyName == 'Žilková'")

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_EndDate,Employment_Ratio,Guarantee,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy
17361,10056354153,Žilková,Viktória,NaN,Mgr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
32149,14058627798,Žilková,Viktória,NaN,NaN,MSc.,Univerzita Komenského v Bratislave,Lekárska fakulta,Výskumný pracovník - výskumník,s vysokoškolským vzdelaním druhého stupňa,2022-06-15,NaN,50,False,NaN,NaN,NaN,NaN,NaN


... V tomto bode sme zistili, že naše stiahnuté xml zrejme obsahuje len dáta pre aktuálnych zamestnancov...

Pre minulých (zamestnancov) neobsahuje údaje o zamestnaní, predovšetkým nevieme z dát zistiť, v akom období a kde boli zamestnaní, hoci ich meno je v dátach.

In [23]:
df.query("Employment_EndDate.isna()")

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_EndDate,Employment_Ratio,Guarantee,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy
0,18799121690,Kočan,Štefan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,NaN,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,NaN,100,False,NaN,NaN,NaN,NaN,NaN
1,14002860472,Korgo,Dušan,DOC,JUDR,PHD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
2,17922733804,Buzalka,Ján,prof.,PhDr.,CSc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
3,16290288493,Stieranka,Jozef,prof.,Ing.,PhD.,Akadémia Policajného zboru,NaN,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,NaN,100,True,"['bezpečnostnoprávna ochrana osôb a majetku', ...","['Akadémia Policajného zboru', 'Akadémia Polic...","[None, None, None, None, None, None, None]","['1', '2', '1', '2', '2', '1', '2']","['1', '1', '2', '2', '2', '3', '3']"
4,21401303068,Wäldl,Ľuboš,doc.,JUDr.,PhD.,Akadémia Policajného zboru,NaN,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,NaN,100,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Spurný,Joža,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,NaN,100,False,NaN,NaN,NaN,NaN,NaN
32726,13500390262,Harčárová,Marcela,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,NaN,50,False,NaN,NaN,NaN,NaN,NaN
32727,17108478442,Juhásová,Jana,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,NaN,50,False,NaN,NaN,NaN,NaN,NaN
32728,15917135755,Kubušová,Miriama,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,NaN,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,NaN,50,False,NaN,NaN,NaN,NaN,NaN


-> Až 32434 riadkov nemá EndDate, čo aj potvrdzuje našu hypotézu, že máme zlé dáta :(